# Applied Machine Learning Systems (MLS-1) - ELEC0134

# Final Assignment - Classification of Tumours

## Task A - Binary Classification using Logistic Regression

## Completed by Student Number - 18014580

This is a Jupyter Notebook submitted as part of the final assignment for the Applied Machine Learning Systems (MLS-1) coursework which involves tumor classification and identification.

This particular notebook tests the logistic regression method on Task A to find the accuracy of logistic regression for this particular task.

### Importing different packages

Initially, we must import the different packages needed for this task. The packages required to implement logistic regression for this binary task are pandas, sklearn, numpy and scipy.

In [6]:
import pandas as pd
import numpy as np
# import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skimage.io import imread, imshow
from keras.preprocessing import image

import os

print("All imports carried out successfully")

All imports carried out successfully


### Loading the data from the data sets

In [ ]:
#Loading the CSV file containing the labels
tumour_labels = pd.read_csv('./dataset/label.csv')

### Splitting the data into training and testing data